In [1]:
import pandas as pd
import json
import sqlite3
import os

# Step 1: Load CSV Data
orders_df = pd.read_csv('orders.csv')

# Step 2: Load JSON Data
with open('users.json', 'r') as f:
    users_data = json.load(f)
users_df = pd.DataFrame(users_data)

# Step 3: Load SQL Data
# The .sql file contains CREATE TABLE and INSERT INTO statements.
# We'll use sqlite3 to execute these and then read the table.
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

with open('restaurants.sql', 'r') as f:
    sql_script = f.read()
    # sqlite3.executescript can handle multiple statements
    cursor.executescript(sql_script)

restaurants_df = pd.read_sql_query("SELECT * FROM restaurants", conn)
conn.close()

# Inspecting columns to handle overlaps before merging
print("Orders columns:", orders_df.columns.tolist())
print("Users columns:", users_df.columns.tolist())
print("Restaurants columns:", restaurants_df.columns.tolist())

# Step 4: Merge the Data
# orders.user_id -> users.user_id
# orders.restaurant_id -> restaurants.restaurant_id
# Join Type: Left Join

# First merge: orders + users
merged_df = orders_df.merge(users_df, on='user_id', how='left', suffixes=('', '_user'))

# Second merge: merged_df + restaurants
final_df = merged_df.merge(restaurants_df, on='restaurant_id', how='left', suffixes=('', '_rest'))

# Step 5: Create Final Dataset
final_df.to_csv('final_food_delivery_dataset.csv', index=False)

print("\nFinal Dataset Info:")
print(final_df.info())
print("\nFirst few rows of final dataset:")
print(final_df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'orders.csv'

In [ ]:
import pandas as pd

# Load the merged dataset
df = pd.read_csv('final_food_delivery_dataset.csv')

# Convert order_date to datetime
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)

# 1. Revenue by City
revenue_by_city = df.groupby('city')['total_amount'].sum().sort_values(ascending=False)

# 2. Revenue by Cuisine
revenue_by_cuisine = df.groupby('cuisine')['total_amount'].sum().sort_values(ascending=False)

# 3. Membership Impact (Average Order Value)
membership_stats = df.groupby('membership')['total_amount'].agg(['mean', 'count', 'sum'])

# 4. Seasonality (Revenue by Month)
df['month'] = df['order_date'].dt.month
revenue_by_month = df.groupby('month')['total_amount'].sum()

# 5. Top 5 Restaurants by Revenue
top_restaurants = df.groupby('restaurant_id')['total_amount'].sum().sort_values(ascending=False).head(5)

# 6. Total Orders and Revenue
total_orders = len(df)
total_revenue = df['total_amount'].sum()

print("Revenue by City:\n", revenue_by_city)
print("\nRevenue by Cuisine:\n", revenue_by_cuisine)
print("\nMembership Stats:\n", membership_stats)
print("\nRevenue by Month:\n", revenue_by_month)
print("\nTop 5 Restaurants by Revenue:\n", top_restaurants)
print(f"\nTotal Orders: {total_orders}")
print(f"Total Revenue: {total_revenue}")